In [212]:
import numpy as np
import pandas as pd

In [213]:
# Import the data
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
stores = pd.read_csv('../data/stores.csv')
transactions = pd.read_csv('../data/transactions.csv')
oil = pd.read_csv('../data/oil.csv')
holidays = pd.read_csv('../data/holidays_events.csv')

# Dictionary for all datasets
datasets = {'train':train, 'test':test, 'stores':stores, 'transactions':transactions, 'oil':oil, 'holidays':holidays}

# Cleaning

### Standarizing dates

We convert any dates in the datasets to pandas Timestamps.

In [214]:
for df in iter(datasets.values()):
    if 'date' in df.columns:
        df['date'] = pd.to_datetime( df['date'] )

### Training/Testing Set

We drop the `id` column from the training/testing set. The preliminary analysis proved this is just a redundant row indexer.

In [215]:
train = train.drop('id',axis=1)
test = test.drop('id',axis=1)

### Oil Dataset

We change `dcoilwtico` to just `oil` for simplicity.

In [216]:
oil = oil.rename({'dcoilwtico': 'oil'}, axis=1)

The preliminary analysis shows 43 missing daily oil price values and missing weekend oil price values. 

We add the weekend days first as more null values, then interpolate 

Also, the oil price for the very first day (2013-01-01) is missing. We can manually add that here using the oil prices from 2012-12-31 and 2013-01-02 from (https://fred.stlouisfed.org/data/DCOILWTICO). We separately verified that is safe to do, since these oil prices match the ones in the oil dataset.

In [217]:
# Manually add oil price for the first day using average of 2012-12-31 and 2013-01-02 oil prices
oil.iloc[0,1] = 92.485                                  

In [218]:
# Create DataFrame with all dates in desired range, including weekends
dates = pd.DataFrame(pd.date_range(start='1/1/2013', end='8/31/2017',freq='D'), columns=['date'])
# Merge with oil data set, so that weekend dates are added to oil with null values
oil = dates.merge(oil,how='left', on='date')
# Interpolate all missing values in oil (all but possibly one of the gaps are of size 1,2, or 3)
oil['oil'] = oil['oil'].interpolate()

### Holiday Dataset

Rename the `type` column so that it won't conflict with store type.

In [219]:
holidays = holidays.rename({'type':'hol_type'},axis='columns')

From the preliminary analysis, two transferred holidays need their description updated so that all tranfer holidays have consistent formatting.

In [220]:
holidays.loc[304,'description'] = 'Traslado Fundacion de Cuenca'
holidays.loc[329, 'description'] = 'Traslado Fundacion de Ibarra'

There were also a couple mislabelled Additional Holidays, and one that should be deleted for redundancy.

In [221]:
holidays.loc[182,'type'] = 'Additional'
holidays.loc[322,'type'] = 'Holiday'
holidays = holidays.drop(264, axis=0)

We separate the holiday `locale` variable into three columns (local, regional, national).

In [222]:
holidays = pd.get_dummies(holidays, columns=['locale'],prefix='Hol')
#We can adjust weights here or later
holidays['Hol_Local']=holidays['Hol_Local']*1
holidays['Hol_National']=holidays['Hol_National']*1
holidays['Hol_Regional']=holidays['Hol_Regional']*1

In [223]:
# Looking at local holidays
hol_loc=holidays[holidays['Hol_Local']==1]
hol_loc=hol_loc.rename(columns={'locale_name':'city'})
hol_loc=hol_loc[['date', 'hol_type', 'city', 'description', 'transferred','Hol_Local']]
hol_loc.sample()

,date,hol_type,city,description,transferred,Hol_Local
197,2015-11-12,Holiday,Ambato,Independencia de Ambato,False,1


In [224]:
# Looking at Regional holidays
hol_reg=holidays[holidays['Hol_Regional']==1]
hol_reg=hol_reg.rename(columns={'locale_name':'state'})
hol_reg=hol_reg[['date', 'hol_type', 'state', 'description', 'transferred', 'Hol_Regional']]
hol_reg.sample()

,date,hol_type,state,description,transferred,Hol_Regional
279,2016-11-07,Holiday,Santa Elena,Provincializacion Santa Elena,False,1


In [225]:
# Looking at National holidays
hol_nat=holidays[holidays['Hol_National']==1]
hol_nat=hol_nat[['date', 'hol_type', 'description', 'transferred', 'Hol_National']]
hol_nat.sample()

,date,hol_type,description,transferred,Hol_National
157,2014-12-26,Additional,Navidad+1,False,1


In [226]:
# Create a map of national holidays
holiday_nat_map = dict(zip(hol_nat['date'], hol_nat['Hol_National']))
holiday_nat_type_map = dict(zip(hol_nat['date'], hol_nat['hol_type']))
holiday_nat_name_map = dict(zip(hol_nat['date'], hol_nat['description']))
holiday_nat_transf_map = dict(zip(hol_nat['date'], hol_nat['transferred']))

# Create a map of regional holidays
holiday_reg_map = dict(zip(zip(hol_reg['date'], hol_reg['state'].str.strip()), hol_reg['Hol_Regional']))
holiday_reg_type_map = dict(zip(zip(hol_reg['date'], hol_reg['state'].str.strip()), hol_reg['hol_type']))
holiday_reg_name_map = dict(zip(zip(hol_reg['date'], hol_reg['state'].str.strip()), hol_reg['description']))
holiday_reg_transf_map = dict(zip(hol_reg['date'], hol_reg['transferred']))

# Create a map of local holidays
holiday_loc_map = dict(zip(zip(hol_loc['date'], hol_loc['city'].str.strip()), hol_loc['Hol_Local']))
holiday_loc_type_map = dict(zip(zip(hol_loc['date'], hol_loc['city'].str.strip()), hol_loc['hol_type']))
holiday_loc_name_map = dict(zip(zip(hol_loc['date'], hol_loc['city'].str.strip()), hol_loc['description']))
holiday_loc_transf_map = dict(zip(zip(hol_loc['date'], hol_loc['city'].str.strip()), hol_loc['transferred']))

# Merging

We first inner join the store and transaction data along the `store_nbr` column.

This guarantees no duplicates or new null values.

In [227]:
# Merge stores with transactions on date and store_nbr
X = stores.merge(transactions, how='inner', on='store_nbr')
X = X.sort_values(by=['date','store_nbr'],axis=0).reset_index(drop=True)
X = X[['date','store_nbr','type','cluster','city','state','transactions']]

Before merging more data, we break down the `date` column into year, month, w|eek, day, and day of week.

In [228]:
X = X.assign(**{'year': pd.Series( [X.date[i].year for i in X.index]), 
            'month': pd.Series( [X.date[i].month for i in X.index]), 
            'week_number': pd.Series( [X.date[i].week for i in X.index]), 
            'day':pd.Series( [X.date[i].day for i in X.index]), 
            'day_of_week': pd.Series( [X.date[i].dayofweek for i in X.index]) })
X = X[['date','year', 'month', 'week_number', 'day', 'day_of_week','store_nbr','type','cluster','city', 'state', 'transactions']]

Next, we  join oil prices along the `date` column.

Since there is one oil price per date and we filled null values in oil, this won't give duplicates or new null values.

In [229]:
X = X.merge(oil, how='left', on='date')
X = X.sort_values(by=['date','store_nbr'],axis=0).reset_index(drop=True)

Finally, we inner join our training data along both `date` and `store_nbr`.

In [230]:
X = X.merge(train, how='left', on=['date','store_nbr'])

Here we join the holiday data using the mappings defined earlier.

In [231]:
# Add empty tranferred and holiday type columns
X['transferred'] = np.nan
X['hol_type'] = np.nan

#Use mappings to fill in the values for national holidays
X['hol_Nat'] = X['date'].map(holiday_nat_map)
X.loc[X['transferred'].isna(), 'transferred'] = X.loc[X['transferred'].isna(), 'date'].map(holiday_nat_transf_map)
X['hol_Nat_name'] = X['date'].map(holiday_nat_name_map)
X.loc[X['hol_type'].isna(), 'hol_type'] = X['date'].map(holiday_nat_type_map)

# Assign regional holidays based on mapping 
X['hol_Reg'] = X.apply(lambda row: holiday_reg_map.get((row['date'], row['state'])), axis=1)
X.loc[X['transferred'].isna(), 'transferred'] = X.loc[X['transferred'].isna(), 'date'].map(holiday_reg_transf_map)
X['hol_Reg_name'] = X.apply(lambda row: holiday_reg_name_map.get((row['date'], row['state'])), axis=1)
X.loc[X['hol_type'].isna(), 'hol_type'] = X.loc[X['hol_type'].isna()].apply(lambda row: holiday_reg_type_map.get((row['date'], row['state'])), axis=1)

# Assign local holidays based on mapping 
X['hol_Loc'] = X.apply(lambda row: holiday_loc_map.get((row['date'], row['city'])), axis=1)
X.loc[X['transferred'].isna(), 'transferred'] = X.loc[X['transferred'].isna(), 'date'].map(holiday_nat_transf_map)

X['transferred'] = X.apply(
    lambda row: holiday_loc_map.get((row['date'], row['city'])) if pd.isna(row['transferred']) else row['transferred'], axis=1
)
X.loc[X['hol_type'].isna(), 'hol_type']= X.loc[X['hol_type'].isna()].apply(lambda row: holiday_loc_type_map.get((row['date'], row['city'])), axis=1)
X['hol_loc_name'] = X.apply(lambda row: holiday_loc_name_map.get((row['date'], row['city'])), axis=1)

# Just fillna
X[['hol_Nat','hol_Reg','hol_Loc']]=X[['hol_Nat','hol_Reg','hol_Loc']].fillna(0)


/var/folders/26/840b8l353yv58xh3z45hs1tc0000gn/T/ipykernel_43727/418294253.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[False False False ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X.loc[X['transferred'].isna(), 'transferred'] = X.loc[X['transferred'].isna(), 'date'].map(holiday_nat_transf_map)
/var/folders/26/840b8l353yv58xh3z45hs1tc0000gn/T/ipykernel_43727/418294253.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Holiday' 'Holiday' 'Holiday' ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X.loc[X['hol_type'].isna(), 'hol_type'] = X['date'].map(holiday_nat_type_map)


Reorder columns and create boolean columns for each holiday type.

In [232]:
#Separating types of the holidays 
X = pd.get_dummies(X,columns=['hol_type'], prefix='hol_type')

In [233]:
#Reorder columns
X = X[['date', 'year', 'month', 'week_number', 'day', 'day_of_week',
       'store_nbr', 'type', 'cluster', 'city', 'state', 'transactions', 'oil',
       'hol_Nat', 'hol_Nat_name', 'hol_Reg', 'hol_Reg_name', 'hol_Loc','hol_loc_name', 
       'transferred','hol_type_Additional', 'hol_type_Bridge', 'hol_type_Event',
       'hol_type_Holiday', 'hol_type_Transfer', 'hol_type_Work Day',
       'family', 'onpromotion', 'sales']] 

# Merging (Alternative)

This approach to merging starts with the training set, and ensures no rows of the training data are lost. However, as a result of NaN values and missing dates among the other data sets, this might contain some NaN values.

Start with DataFrames containing all dates in the desired time frames.

In [234]:
# DataFrame with all the days
train_dates = pd.DataFrame(pd.date_range(start='1/1/2013', end='8/15/2017',freq='D'), columns=['date'])
test_dates = pd.DataFrame(pd.date_range(start='8/16/2017', end='8/31/2017',freq='D'), columns=['date'])

# Breaking down dates into year, month, day, etc.
train_dates = train_dates.assign(**{'year': pd.Series( [train_dates.date[i].year for i in train_dates.index]), 
            'month': pd.Series( [train_dates.date[i].month for i in train_dates.index]), 
            'week_number': pd.Series( [train_dates.date[i].week for i in train_dates.index]), 
            'day':pd.Series( [train_dates.date[i].day for i in train_dates.index]), 
            'day_of_week': pd.Series( [train_dates.date[i].dayofweek for i in train_dates.index]) })
test_dates = test_dates.assign(**{'year': pd.Series( [test_dates.date[i].year for i in test_dates.index]), 
            'month': pd.Series( [test_dates.date[i].month for i in test_dates.index]), 
            'week_number': pd.Series( [test_dates.date[i].week for i in test_dates.index]), 
            'day':pd.Series( [test_dates.date[i].day for i in test_dates.index]), 
            'day_of_week': pd.Series( [test_dates.date[i].dayofweek for i in test_dates.index]) })

# Reorder columns
train_dates = train_dates[['date','year', 'month', 'week_number', 'day', 'day_of_week']]
test_dates = test_dates[['date','year', 'month', 'week_number', 'day', 'day_of_week']]


Add oil price for each day.

In [235]:
# Add oil price for each day
X_alt = oil.merge(train_dates, how='inner', on='date')
y_alt = oil.merge(test_dates, how='inner', on='date')

Left join training set data (left) with date/oil (right) along the `date` column.

This ensures all training data is kept, while the date DataFrame drops December 25 2013, 2014, 2015, 2016, which are missing from train.csv.

In [236]:
X_alt = train.merge(X_alt, how='left', on='date')           # 3,000,888 rows
y_alt = test.merge(y_alt, how='left', on='date')            # 28,512 rows

Left join merged data (left) with stores (right) along the `store_nbr` column.

In [237]:
X_alt = X_alt.merge(stores, how='left', on='store_nbr')     # 3,000,888 rows
y_alt = y_alt.merge(stores, how='left', on='store_nbr')     # 28,512 rows

Left join merged date (left) with transactions along `date` then `store_nbr`.

Transactions is additionally missing data for Jan 1 and Jan 3 of 2016.

In [238]:
# Note: the testing set has no transaction data, so we can't add that here
X_alt = X_alt.merge(transactions, how='left', on=['date','store_nbr'])

Finally we add the holiday data using the mappings we constructed.

In [239]:
# Add empty transferred and holiday type columns
X_alt['transferred'] = np.nan
X_alt['hol_type'] = np.nan

#Use mappings to fill in the values for national holidays
X_alt['hol_Nat'] = X_alt['date'].map(holiday_nat_map)
X_alt.loc[X_alt['transferred'].isna(), 'transferred'] = X_alt.loc[X_alt['transferred'].isna(), 'date'].map(holiday_nat_transf_map)
X_alt['hol_Nat_name'] = X_alt['date'].map(holiday_nat_name_map)
X_alt.loc[X_alt['hol_type'].isna(), 'hol_type'] = X_alt['date'].map(holiday_nat_type_map)

# Assign regional holidays based on mapping 
X_alt['hol_Reg'] = X_alt.apply(lambda row: holiday_reg_map.get((row['date'], row['state'])), axis=1)
X_alt.loc[X_alt['transferred'].isna(), 'transferred'] = X_alt.loc[X_alt['transferred'].isna(), 'date'].map(holiday_reg_transf_map)
X_alt['hol_Reg_name'] = X_alt.apply(lambda row: holiday_reg_name_map.get((row['date'], row['state'])), axis=1)
X_alt.loc[X_alt['hol_type'].isna(), 'hol_type'] = X_alt.loc[X_alt['hol_type'].isna()].apply(lambda row: holiday_reg_type_map.get((row['date'], row['state'])), axis=1)

# Assign local holidays based on mapping 
X_alt['hol_Loc'] = X_alt.apply(lambda row: holiday_loc_map.get((row['date'], row['city'])), axis=1)
X_alt.loc[X_alt['transferred'].isna(), 'transferred'] = X_alt.loc[X_alt['transferred'].isna(), 'date'].map(holiday_nat_transf_map)

X_alt['transferred'] = X_alt.apply(
    lambda row: holiday_loc_map.get((row['date'], row['city'])) if pd.isna(row['transferred']) else row['transferred'], axis=1
)
X_alt.loc[X_alt['hol_type'].isna(), 'hol_type']= X_alt.loc[X_alt['hol_type'].isna()].apply(lambda row: holiday_loc_type_map.get((row['date'], row['city'])), axis=1)
X_alt['hol_loc_name'] = X_alt.apply(lambda row: holiday_loc_name_map.get((row['date'], row['city'])), axis=1)

# Just fillna
X_alt[['hol_Nat','hol_Reg','hol_Loc']]=X_alt[['hol_Nat','hol_Reg','hol_Loc']].fillna(0)

/var/folders/26/840b8l353yv58xh3z45hs1tc0000gn/T/ipykernel_43727/3630162675.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[False False False ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_alt.loc[X_alt['transferred'].isna(), 'transferred'] = X_alt.loc[X_alt['transferred'].isna(), 'date'].map(holiday_nat_transf_map)
/var/folders/26/840b8l353yv58xh3z45hs1tc0000gn/T/ipykernel_43727/3630162675.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Holiday' 'Holiday' 'Holiday' ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_alt.loc[X_alt['hol_type'].isna(), 'hol_type'] = X_alt['date'].map(holiday_nat_type_map)


In [240]:
#Separating types of the holidays 
X_alt = pd.get_dummies(X_alt,columns=['hol_type'], prefix='hol_type')

In [241]:
# Reorder columns
X_alt = X_alt[['date', 'year', 'month', 'week_number', 'day', 'day_of_week', 
       'store_nbr','type', 'cluster', 'city', 'state','transactions','oil',
       'hol_Nat','hol_Nat_name',  'hol_Reg','hol_Reg_name','hol_Loc','hol_loc_name',
       'transferred','hol_type_Additional','hol_type_Bridge', 'hol_type_Event',
       'hol_type_Holiday', 'hol_type_Transfer', 'hol_type_Work Day',
       'family', 'onpromotion', 'sales']] 

# Creating csv

In [242]:
X.sample(5) 

,date,year,month,week_number,day,day_of_week,store_nbr,type,cluster,city,...,transferred,hol_type_Additional,hol_type_Bridge,hol_type_Event,hol_type_Holiday,hol_type_Transfer,hol_type_Work Day,family,onpromotion,sales
1282946,2015-04-11,2015,4,15,11,5,13,C,15,Latacunga,...,None,False,False,False,False,False,False,BREAD/BAKERY,2,407.000
2640467,2017-06-12,2017,6,24,12,0,37,D,2,Cuenca,...,None,False,False,False,False,False,False,BREAD/BAKERY,0,695.903
593946,2014-01-25,2014,1,4,25,5,9,B,6,Quito,...,None,False,False,False,False,False,False,GROCERY I,0,5452.000
2633785,2017-06-08,2017,6,23,8,3,50,A,14,Ambato,...,None,False,False,False,False,False,False,"LIQUOR,WINE,BEER",2,88.000
2483934,2017-03-16,2017,3,11,16,3,10,C,15,Quito,...,None,False,False,False,False,False,False,MEATS,18,220.933


In [243]:
X_alt.sample(5)

,date,year,month,week_number,day,day_of_week,store_nbr,type,cluster,city,...,transferred,hol_type_Additional,hol_type_Bridge,hol_type_Event,hol_type_Holiday,hol_type_Transfer,hol_type_Work Day,family,onpromotion,sales
1042997,2014-08-10,2014,8,32,10,6,23,D,9,Ambato,...,False,False,False,False,True,False,False,SEAFOOD,0,15.778
475810,2013-09-25,2013,9,39,25,2,1,D,13,Quito,...,None,False,False,False,False,False,False,HOME AND KITCHEN II,0,0.000
1182063,2014-10-27,2014,10,44,27,0,26,D,10,Guayaquil,...,None,False,False,False,False,False,False,BEVERAGES,1,818.000
299378,2013-06-18,2013,6,25,18,1,1,D,13,Quito,...,None,False,False,False,False,False,False,BEAUTY,0,4.000
1317853,2015-01-12,2015,1,3,12,0,35,C,3,Playas,...,None,False,False,False,False,False,False,SCHOOL AND OFFICE SUPPLIES,0,0.000


In [244]:
X.to_csv("../data/merged_train.csv", index = False)
X_alt.to_csv("../data/merged_train_alt.csv", index = False)